In [5]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
wpt = nltk.tokenize.TweetTokenizer()

In [38]:
def list_counter_sum(counter_list):
    counter_final = Counter()
    [counter_final.update(x) for x in counter_list]
    return counter_final

def ngram_counter(doc_list, ngram, tokenizer):
    vectorizer = CountVectorizer(ngram_range=(ngram,ngram), tokenizer=tokenizer)
    analyzer = vectorizer.build_analyzer()
    tokenized_list = [analyzer(x) for x in doc_list]
    test = [Counter(z) for z in tokenized_list]
    return {ngram: sum(test, Counter())}

def ngram_counter_summary(doc_list, min_ngram, max_ngram, tokenizer, num_process=1, parallel=False):
    
    if parallel:
        assert num_process is not None, "Please specify number of processes!"
        main_doc_list = np.array_split(doc_list, num_process)
        ngram_dict_list = []
        for i in range(min_ngram,max_ngram + 1):
            ngram_counter_dict_list = Parallel(n_jobs=num_process)(delayed(ngram_counter)(x,i, tokenizer) for x in main_doc_list)
            ngram_dict_list = ngram_dict_list + ngram_counter_dict_list
            
        a={}
        for d in ngram_dict_list:
            for k,v in d.items():
                try:
                    a[k].append(v)
                except KeyError:
                    a[k]=[v]
                    
        ngram_counter_dict = dict((k,list_counter_sum(v)) for k,v in a.items())
#         ngram_counter_dict = a
        
    else:
        ngram_counter_dict = {}
        for i in range(min_ngram, max_ngram + 1):
            ngram_counter_dict.update(ngram_counter(doc_list, i, tokenizer))
    
    return ngram_counter_dict

In [7]:
string_list = [
    'This is an example',
    'This is just an example',
    'This is just a fucking example',
    'This is just a fucking noob example',
    'This is just a damn fucking noob example'
]

In [8]:
ngram_counter_summary(string_list,1,3,wpt.tokenize)

{1: Counter({'this': 5,
          'is': 5,
          'an': 2,
          'example': 5,
          'just': 4,
          'a': 3,
          'fucking': 3,
          'noob': 2,
          'damn': 1}),
 2: Counter({'this is': 5,
          'is an': 1,
          'an example': 2,
          'is just': 4,
          'just an': 1,
          'just a': 3,
          'a fucking': 3,
          'fucking example': 1,
          'fucking noob': 1,
          'noob example': 2,
          'fucking damn': 1,
          'damn noob': 1}),
 3: Counter({'this is an': 1,
          'is an example': 1,
          'this is just': 4,
          'is just an': 1,
          'just an example': 1,
          'is just a': 3,
          'just a fucking': 3,
          'a fucking example': 1,
          'a fucking noob': 1,
          'fucking noob example': 1,
          'a fucking damn': 1,
          'fucking damn noob': 1,
          'damn noob example': 1})}

In [39]:
top = 3
ngram = 2
ngram_counter(string_list,ngram,wpt.tokenize)[ngram].most_common()[:top]

[('this is', 5), ('is just', 4), ('just a', 3)]